In [1]:
import pandas as pd

In [89]:
knvv_mdg = pd.read_excel('../sync/c-after-sync/mdg/knvv.xlsx', dtype=str)
knvv_ecc = pd.read_excel('../sync/c-after-sync/ecc/knvv.xlsx', dtype=str)

In [90]:
print(len(knvv_ecc.columns))
print(len(knvv_mdg.columns))

80
80


In [91]:
cols_mdg = set(knvv_mdg.columns.str.lower())
cols_ecc = set(knvv_ecc.columns.str.lower())

In [92]:
only_in_mdg = sorted(cols_mdg - cols_ecc)
only_in_ecc = sorted(cols_ecc - cols_mdg)

In [93]:
df_only_mdg = pd.DataFrame({'In mdg not in ECC': only_in_mdg})
df_only_ecc = pd.DataFrame({'In ECC not in mdg': only_in_ecc})

In [94]:
print(df_only_ecc)
print(df_only_mdg)

      In ECC not in mdg
0  cust credit category
1             geography
2         industry type
  In mdg not in ECC
0  customer group 1
1  customer group 2
2  customer group 3


In [ ]:
with pd.ExcelWriter('../sync/c-after-sync/column_mismatch_knvv.xlsx', engine='openpyxl') as writer:
    df_only_mdg.to_excel(writer, sheet_name='In mdg', index=False)
    df_only_ecc.to_excel(writer, sheet_name='In ECC', index=False)

In [96]:
n_cols_ecc = list(set(knvv_ecc.columns) - (set(knvv_ecc.columns) - set(knvv_mdg.columns)))
n_cols_mdg = list(set(knvv_mdg.columns) - (set(knvv_mdg.columns) - set(knvv_ecc.columns)))

In [73]:
n_cols_ecc.append('Supplier')
n_cols_mdg.append('Vendor')

In [97]:
knvv_ecc = knvv_ecc[n_cols_ecc]
knvv_mdg = knvv_mdg[n_cols_mdg]

In [98]:
print(knvv_ecc.columns)
print(knvv_mdg.columns)

Index(['Overdeliv. Tolerance', 'Exchange Rate Type', 'Product attribute 3',
       'Price determination', 'Order probability', 'Batch split allowed',
       'Paymt guarant. proc.', 'Unit of Measure Grp', 'Incoterms (Part 2)',
       'Part.dlv./item', 'Rebate index', 'Order Combination',
       'Agency business', 'Authorization Group', 'Underdel. Tolerance',
       'Cust.pric.procedure', 'Incoterms Location 2', 'Cost estimate',
       'Switch off rounding', 'Customer Statistics Group', 'Price List',
       'Rebate', 'Item proposal', 'Division', 'Delivering Plant',
       'Product attribute 9', 'Empties Deposits frm', 'Invoice list sched.',
       'Business Purpose Completed Flag', 'Relevant for POD', 'Emp.Update Off',
       'Shipping Conditions', 'Cust.Acct.Assg.Group', 'Product attribute 6',
       'Customer group 5', 'Prod.prop. customer proced.', 'Max. cost estimate',
       'Product attribute 5', 'Product attribute 2', 'Product attribute 8',
       'Carrier is to be notified', 'Com

In [ ]:
knvv_ecc['key'] = knvv_ecc.apply(
    lambda row: 
        str(row['Customer'])
        + str(row['Sales Organization'])
        + str(row['Distribution Channel'])
        + str(row['Division']),
        # + str(row['Sequence Number']), 
        axis=1
)

In [ ]:
knvv_mdg['key'] = knvv_mdg.apply(
    lambda row: 
        str(row['Customer']),
        # + str(row['Company Code']),
        # + str(row['Distribution Channel'])
        # + str(row['Division']),
        # + str(row['Sequence Number']),
    axis=1
)

In [ ]:
print(len(knvv_ecc))
print(len(knvv_mdg))

2417
2403


In [ ]:
knvv_ecc.set_index('key', inplace=True)
knvv_mdg.set_index('key', inplace=True)

In [ ]:
a_knvv_mdg, a_knvv_ecc = knvv_mdg.align(knvv_ecc, join='outer')
# a_knvv_mdg, a_knvv_ecc = knvv_mdg, knvv_ecc


In [ ]:
print(len(a_knvv_ecc))
print(len(a_knvv_mdg))

19727
19727


In [ ]:
diffs = a_knvv_mdg.compare(
    a_knvv_ecc,
    keep_equal=True,      # show all cells
    # keep_equal=False,      # only show changed cells
    keep_shape=False,       # retain full shape if you want blanks where no change
    result_names=('MDG','ECC')
)

In [87]:
diffs.columns = [f"{col[0]} ({col[1]})" for col in diffs.columns]

In [ ]:
diffs.to_excel('../sync/c-after-sync/knvv_allData_changes.xlsx')
# diffs.to_excel('../sync/c-after-sync/knvv_changes.xlsx')

In [73]:
# Ignore

In [ ]:
f = pd.read_excel('../knvvData_changes.xlsx', sheet_name='Sheet3')

In [148]:
f.head()

,Name,email
0,3M Electro & Communication,pbhutani@mmm.com
1,3M Electro & Communication,srjanak@mmm.com
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM
4,AAALAC INTERNATIONAL,msprouse@aaalac.org


In [153]:
def func(df):
    n = df['Name'].split()
    
    for i in n:
        if i.lower() in df['email'].lower():
            return True
    return False

In [154]:
f['new'] = f.apply(lambda row: func(row), axis=1)

In [155]:
f

,Name,email,new
0,3M Electro & Communication,pbhutani@mmm.com,False
1,3M Electro & Communication,srjanak@mmm.com,False
2,A P ELECTRICALS AND ENGINEERS,SVLNRAO@APEEONLINE.COM,True
3,A P ELECTRICALS AND ENGINEERS,VIJAY@APEEONLINE.COM,True
4,AAALAC INTERNATIONAL,msprouse@aaalac.org,True
...,...,...,...
2828,ZELLE BIOTECHNOLOGY PVT LTD,accounts@zellebiotech.com,True
2829,ZELLE BIOTECHNOLOGY PVT LTD,orders@zellebiotech.com,True
2830,ZELLE BIOTECHNOLOGY PVT LTD,service@zellebiotech.com,True
2831,ZoomInfo Technologies LLC,ar@zoominfo.com,True


In [156]:
f.to_csv('email.csv')

In [ ]:
knvvcc.columns

Index(['Address Number', 'from', 'Address Version', 'To', 'Title', 'Name',
       'Name 2', 'Name 3', 'Name 4', 'Converted name (with form of address)',
       'c/o name', 'City', 'District', 'City Code', 'District.1',
       'City (Diff. from Postal City)', 'City Code.1', 'Test stat./City file',
       'Reg. Struct. Grp.', 'Postal Code', 'PO Box Postal Code',
       'Company Postal Code', 'Postl Code Extension', 'Postl Code Extension.1',
       'Postl Code Extension.2', 'PO Box', 'Undeliverable', 'PO Box w/o No.',
       'PO Box City', 'City Code.2', 'PO Box Region', 'PO box country',
       'Delivery District', 'Transportation zone', 'Street', 'Undeliverable.1',
       'Street Code', 'Street Abbreviation', 'House Number', 'Supplement',
       'House Number Range', 'Street 2', 'Street 3', 'Street 4', 'Street 5',
       'Building Code', 'Floor', 'Room Number', 'Country Key', 'Language Key',
       'Region', 'Address group', 'Flag: There are more address group assig',
       'personal a

In [ ]:
knvv.apply(lambda row: '&' in str(row['Street']), axis=1).value_counts()

False    2944
True       78
Name: count, dtype: int64